# INTRO
we map a dataset with the dictionary and also extract all relevant information for graph construction

# HEADER
adding a link to a shared folder where to find the ngram class

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../shared/'))

# IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from NGram_KNrealRecursion import recursive_NGramKneserNey
import pickle
import networkx as nx

In [ ]:
nx.__version__

In [ ]:
from imports.general_settings import *
from imports.general_functions import *

In [ ]:
%load_ext autoreload
%autoreload 2

# GENERAL SETTINGS

In [ ]:
projectName

In [ ]:
print_valid_relative_positions(valid_relative_positions)

In [ ]:
baseFolder = '..\\..\\data\\' + projectName + '\\'

In [ ]:
inputFolder_data = "00_raw_data"
inputFolder_dictionary = "01_activity_dictionary"

In [ ]:
outputFolder = "02_mapped_dataset"

In [ ]:
outputSubfolder = str(len(valid_relative_positions['lateral'])-1) + "x" + str(len(valid_relative_positions['longitudinal'])-1)

In [ ]:
outputFilename_dataset = "mappedDataset"
outputFilename_nGramGraph = "nGramGraph"
outputFilename_networkX = "networkx"

In [ ]:
dictionaryFilename = "activityDictionary_" + str(len(valid_relative_positions['lateral'])-1) + "x" + str(len(valid_relative_positions['longitudinal'])-1) + ".csv"

In [ ]:
dictionaryFilename

In [ ]:
dictionary_indexColumnName = "index"

In [ ]:
inputPath_data = baseFolder + inputFolder_data + "\\"
inputPath_dictionary = baseFolder + inputFolder_dictionary + "\\"

In [ ]:
outputPath = baseFolder + outputFolder + "\\" + outputSubfolder + "\\"

In [ ]:
outputPath

In [ ]:
if outputFolder not in os.listdir(baseFolder) :
    print("creating", baseFolder + outputFolder)
    os.mkdir(baseFolder + outputFolder)

In [ ]:
if outputSubfolder not in os.listdir(baseFolder + outputFolder) :
    print("creating", outputPath)
    os.mkdir(outputPath)

# LOAD TAGGED DATASET

In [ ]:
fileList = os.listdir(inputPath_data)

In [ ]:
fileList

# <a id='meh'> file to be considere</a>

In [ ]:
inputFile = "20170524_PP_02_Run_3.hdf5"

In [ ]:
inputFile

In [ ]:
s = pd.HDFStore(inputPath_data + inputFile)
tagged_dataset = s.get('df')
s.close()

In [ ]:
tagged_dataset.shape

In [ ]:
tagged_dataset.reset_index(inplace=True, drop=True)

In [ ]:
tagged_dataset.head()

In [ ]:
valuesPerCol = {}
colsWithRawValues = []
lastTargetID = 0
for c in tagged_dataset.columns :
    if tagged_dataset[c].dtype == float :
        colsWithRawValues.append(c)
        continue
    cName = c
    if 'host' in c[:len('host')] :
        if c not in valuesPerCol :
            valuesPerCol[c] = []
    elif 'target' in c[:len('target')] :
        targetId = int(c.split('_')[1])
        if targetId > lastTargetID :
            lastTargetID = targetId
        last = '_'.join(c.split('_')[2:])
        cName = 'target_' + last
        if cName not in valuesPerCol :
            valuesPerCol[cName] = []
    uniq = list(tagged_dataset[c].drop_duplicates())
    valuesPerCol[cName] = list(set(valuesPerCol[cName] + uniq))
print("categorical columns and values:")
for k in valuesPerCol :
    print("\t", k, valuesPerCol[k])
print("\ncontinuous columns:")
for k in colsWithRawValues :
    print("\t", k)

In [ ]:
n_targets

# LOAD DICTIONARY

In [ ]:
inputPath_dictionary

In [ ]:
dictionary = pd.read_pickle(inputPath_dictionary + dictionaryFilename)

In [ ]:
dictionary.shape

# TAGGED DATAFRAME CORRECTION BASED ON VALID RELATIVE POSITIONS

In [ ]:
corrected_dataset = tagged_dataset.copy()

In [ ]:
for t in range(n_targets) :
    corrected_dataset.loc[(~corrected_dataset['target_' + str(t) + '_relative_position_longitudinal'].isin(valid_relative_positions['longitudinal'])) |
              (~corrected_dataset['target_' + str(t) + '_relative_position_lateral'].isin(valid_relative_positions['lateral'])),
                        ['target_' + str(t) + '_lateral', 
                         'target_' + str(t) + '_longitudinal',
                         'target_' + str(t) + '_relative_position_longitudinal', 
                         'target_' + str(t) + '_relative_position_lateral',
                         'target_' + str(t) + '_velocity']] = ''

# DATASET MAPPING
we assume that the dataframe has already been parsed in the previous step - dictionary_update

In [ ]:
mapped_dataset = corrected_dataset.copy()

In [ ]:
mapped_dataset.shape

In [ ]:
shared_columns = []
for c in mapped_dataset.columns :
    if c in dictionary.columns :
        shared_columns.append(c)
shared_columns

In [ ]:
mapped_dataset = mapped_dataset.merge(dictionary, on=shared_columns)

In [ ]:
mapped_dataset.sort_values(by=['timestamp'], ascending=[True], inplace=True)

In [ ]:
corrected_dataset.shape, mapped_dataset.shape, dictionary.shape

In [ ]:
sum(np.asarray(list(corrected_dataset['timestamp'])) == np.asarray(list(mapped_dataset['timestamp']))) == len(corrected_dataset)

In [ ]:
fig = plt.figure()
plt.plot(list(mapped_dataset['index']))
plt.show()

# CLEAN UP MAPPED DATASET
so basically remove all images, leave "index" but rename it "dictionary_index"

In [ ]:
cols_to_remove = []
for c in mapped_dataset.columns :
    if c not in [dictionary_indexColumnName, 'n_objects'] + list(corrected_dataset.columns) :
        cols_to_remove.append(c)

cols_to_remove

In [ ]:
mapped_dataset.drop(cols_to_remove, axis=1, inplace=True)

In [ ]:
mapped_dataset.rename(columns={dictionary_indexColumnName : 'dictionary_index'}, inplace=True)

In [ ]:
for c in mapped_dataset.columns :
    print(c)

# SAVING THE MAPPED DATASET

In [ ]:
outputPath

In [ ]:
mapped_dataset.dtypes

In [ ]:
outputFilename_dataset = outputFilename_dataset + "_" + inputFile[:inputFile.find(".hdf5")] + ".csv"

In [ ]:
outputFilename_dataset

In [ ]:
outputPath + outputFilename_dataset

In [ ]:
mapped_dataset.to_csv(outputPath + outputFilename_dataset, index=False)

# N-GRAM GRAPH CREATION
we build a unique ngram, the rendering/filtering should occur later

In [ ]:
model = recursive_NGramKneserNey("world_model_" + str(len(valid_relative_positions['lateral'])-1) + "x" + str(len(valid_relative_positions['longitudinal'])-1), n=nGram_n)

In [ ]:
string = "-".join([str(x) for x in list(mapped_dataset['dictionary_index'])])

In [ ]:
model.feed(string)

In [ ]:
outputFilename_nGramGraph += "_" + inputFile[:inputFile.find(".hdf5")] + ".p"

In [ ]:
outputFilename_nGramGraph

In [ ]:
pickle.dump(model, open(outputPath + outputFilename_nGramGraph, 'wb'))

# NETWORKX CREATION
i.e. a network from nGramGraph without self loops and <start><stop><unk>

In [ ]:
G = nx.DiGraph()

for edge in model.grams[2] :
    n0 = edge[0]
    n1 = edge[1]
    if n0 in ["<start>", "<stop>", "<unk>"] or n1 in ["<start>", "<stop>", "<unk>"] or n0 == n1:
        if n0 != n1 :
            print(edge)
        continue
    G.add_edge(n0, n1)
print(".")

In [ ]:
G.number_of_edges()

In [ ]:
G.number_of_nodes()

In [ ]:
outputFilename_networkX += "_" + inputFile[:inputFile.find(".hdf5")] + ".p"

In [ ]:
outputFilename_networkX

In [ ]:
pickle.dump(G, open(outputPath + outputFilename_networkX, "wb"))

In [ ]:
print(".")

# <a href='#meh'> back </a>